# Charge stability measurements

## Introduction
In this tutorial we will demonstrate how to get started measuring a device with spinQICK!  This requires the dc voltage source you set up in the previous tutorial notebook `01_dc_voltage_source_setup.ipynb`, and the config files created in `00_getting_started.ipynb`.

In [ ]:
import Pyro4
from qick import QickConfig
from spinqick.experiments import tune_electrostatics
from spinqick.helper_functions import hardware_manager

Connect to your qick board using Pyro, more information on setting up and using Pyro with QICK can be found [here](https://github.com/openquantumhardware/qick/tree/main/pyro4).

In [ ]:
Pyro4.config.SERIALIZER = "pickle"
Pyro4.config.PICKLE_PROTOCOL_VERSION = 4

ns_host = "192.168.2.99"
ns_port = 8888
proxy_name = "myqick"

ns = Pyro4.locateNS(host=ns_host, port=ns_port)
soc = Pyro4.Proxy(ns.lookup(proxy_name))
soccfg = QickConfig(soc.get_cfg())
print(soccfg)

## Initialize the TuneElectrostatics class
We start with initializing our voltage source, and then initializing the `TuneElectrostatics` experiment class.  Now we have access to the suite of electrostatics experiments in this class.

In [ ]:
dummy_vsource = hardware_manager.DummyDCSource()
charge = tune_electrostatics.TuneElectrostatics(soccfg, soc, dummy_vsource)

We typically begin by sweeping the gate voltages of multiple gates simultaneously to find their turn-on voltages.  Here, we look for the turn-on voltage of the current path through the dot charge sensor.  The `gate_turn_on` function sweeps several voltages together from 0 to the `max_v` parameter. `measure_buffer` adds a delay, in microseconds, from the time the precision DAC steps to the next point, and the RFSoC begins measuring for the next point.

In [ ]:
data = charge.gate_turn_on(gates=["M1", "Z1", "Z2"], max_v=1.2, num_points=100, measure_buffer=200)

![globalturnon](demo_data/1752779578_global_turn_on.png)

After setting each gate to some voltage above the turn-on threshold, we can run the `gate_action` function, which individually sweeps each gate voltage to zero and back up to `max_v`.  This shows us roughly what the turn on voltages are for each gate. 

In [ ]:
data = charge.gate_action(gates=["M1", "Z1", "Z2"], max_v=1.2, num_points=100, measure_buffer=200)

![gateaction](demo_data/1752779650_gate_action.png)

Now we can perform a 2D sweep of the M and Z gates to choose a DCS peak. The voltage range parameters are relative to the current voltage state of the device, so you will need to set approximate voltages for the M and Z gates first.

In [ ]:
charge.vdc.set_dc_voltage(1.3, "M1")
charge.vdc.set_dc_voltage(0.48, "Z1")
charge.vdc.set_dc_voltage(0.48, "Z2")
data = charge.gvg_dc(
    g_gates=(["Z1", "Z2"], ["M1"]), g_range=((-0.05, 0.05, 100), (-0.05, 0.05, 100)), measure_buffer=200
)
# uncomment and run the function below to sweep 'Z1' and 'Z2' in opposite directions

# data= charge.gvg_dc((['Z1', 'Z2'], ['M1']), ((-0.05, 0.05, 100),(-0.05, 0.05, 100)), 200, sweep_direction=([1, -1], [1]))

![tunemz](demo_data/1754583225_gvg_dc.png)

The `retune_dcs` function will perform a small voltage sweep and look for a peak in conductance.  If `set_v` is `True`, the function sets the gate voltage to a value on the side of the peak.

In [ ]:
dcs = charge.retune_dcs(m_dot="M1", m_range=(-0.01, 0.01, 100), measure_buffer=200, set_v=True)

![retune](demo_data/1752782166_retune_m.png)

We can perform voltage sweeps on any of the gates we like.

In [ ]:
data = charge.gvg_dc(
    g_gates=(["P1"], ["P2"]), g_range=((-0.075, 0.075, 100), (-0.075, 0.075, 100)), measure_buffer=200
)
# Uncomment to compensate on the M gate to get better signal after determining the cross-capacitance matrix

# data= charge.gvg_dc((['P1'], ['P2']), ((-0.075, 0.075, 100),(-0.075, 0.075, 100)), 200)

![pvp](demo_data/1754581593_gvg_dc.png)